Linear regression model from scratch

In [16]:
import numpy as np
import torch

In [17]:
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]],dtype='float32')

In [18]:
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [19]:
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)


In [20]:
w=torch.rand(2,3,requires_grad=True)
b=torch.rand(2,requires_grad=True)

In [21]:
inputs,targets,w,b

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]),
 tensor([[0.7875, 0.0579, 0.5037],
         [0.0778, 0.9998, 0.3908]], requires_grad=True),
 tensor([0.5455, 0.0356], requires_grad=True))

In [22]:
def model(x):
  return x @ w.t() +b

In [23]:
preds=model(inputs)
preds

tensor([[ 83.5729,  89.5058],
        [109.5422, 120.1085],
        [106.0334, 163.4423],
        [101.9985,  65.4236],
        [ 95.7027, 128.7390]], grad_fn=<AddBackward0>)

In [24]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Evaluate MSE

In [25]:
def mse(t1,t2):
  diff=t1-t2
  return torch.sum(diff*diff)/ diff.numel()

In [26]:
loss=mse(preds,targets)
loss

tensor(1077.1155, grad_fn=<DivBackward0>)

In [27]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[2227.6787, 1072.1920,  941.8801],
        [1876.4957, 1844.9728, 1112.1504]])
tensor([23.1699, 21.4438])


In [28]:
with torch.no_grad():
  w-=w.grad*1e-5
  b-=b.grad*1e-5


In [30]:
mse(preds,targets)

tensor(1077.1155, grad_fn=<DivBackward0>)

In [31]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Train for multiple epochs

In [38]:
for i in range(100):
  preds=model(inputs)
  loss=mse(preds,targets)
  loss.backward()
  with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5
    w.grad.zero_()
    b.grad.zero_()

In [39]:
preds=model(inputs)
loss=mse(preds,targets)
loss

tensor(25.4740, grad_fn=<DivBackward0>)

In [40]:
preds

tensor([[ 58.9812,  70.9203],
        [ 82.6979,  97.6079],
        [114.6585, 138.9405],
        [ 31.0395,  40.1846],
        [ 97.0696, 111.9677]], grad_fn=<AddBackward0>)

In [41]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Linear regression using PyTorch built-ins

In [42]:
import torch.nn as nn

In [43]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [44]:
from torch.utils.data import TensorDataset

In [45]:
train_ds=TensorDataset(inputs,targets)

In [48]:
train_ds[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [52]:
from torch.utils.data import DataLoader

In [54]:
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)

In [56]:
for xb,yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 68.,  97.,  70.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.]])
tensor([[102., 120.],
        [ 57.,  69.],
        [ 82., 100.],
        [ 56.,  70.],
        [ 20.,  38.]])


In [59]:
model=nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1073,  0.4482,  0.2922],
        [-0.3507,  0.3393, -0.3011]], requires_grad=True)
Parameter containing:
tensor([-0.5527, -0.1296], requires_grad=True)


In [62]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1073,  0.4482,  0.2922],
         [-0.3507,  0.3393, -0.3011]], requires_grad=True),
 Parameter containing:
 tensor([-0.5527, -0.1296], requires_grad=True)]

In [63]:
preds=model(inputs)
preds

tensor([[ 49.8716, -15.9477],
        [ 67.3504, -21.4585],
        [ 85.7835,  -2.6429],
        [ 40.4747, -32.4551],
        [ 70.3280, -12.8345],
        [ 49.5308, -16.6377],
        [ 67.1945, -22.0988],
        [ 86.1830,  -3.2947],
        [ 40.8155, -31.7651],
        [ 70.5128, -12.7848],
        [ 49.7156, -16.5880],
        [ 67.0096, -22.1485],
        [ 85.9394,  -2.0026],
        [ 40.2898, -32.5048],
        [ 70.6688, -12.1445]], grad_fn=<AddmmBackward0>)

In [64]:
import torch.nn.functional as f

In [65]:
loss_fn=f.mse_loss

In [66]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6578.4561, grad_fn=<MseLossBackward0>)


In [68]:
opt=torch.optim.SGD(model.parameters(),lr=1e-5)

FIT utility function


In [76]:
def fit(num_epoch,model,loss_fn,opt,train_dl):
  for i in range(num_epoch):
    for xb,yb in train_dl:
        pred=model(xb)
        loss=loss_fn(pred,yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    if (i+1) %10==0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num_epoch, loss.item()))



In [77]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 67.5748
Epoch [20/100], Loss: 76.3302
Epoch [30/100], Loss: 62.1726
Epoch [40/100], Loss: 60.1464
Epoch [50/100], Loss: 43.0253
Epoch [60/100], Loss: 20.1133
Epoch [70/100], Loss: 30.0594
Epoch [80/100], Loss: 18.1402
Epoch [90/100], Loss: 21.3823
Epoch [100/100], Loss: 28.7029


In [80]:
preds=model(inputs)
preds


tensor([[ 57.5732,  71.2129],
        [ 80.8974,  96.5747],
        [119.9024, 141.0896],
        [ 24.7149,  41.7010],
        [ 97.4930, 109.2345],
        [ 56.3876,  70.0889],
        [ 80.4961,  95.8939],
        [120.0853, 141.2811],
        [ 25.9005,  42.8251],
        [ 98.2773, 109.6779],
        [ 57.1719,  70.5322],
        [ 79.7119,  95.4506],
        [120.3037, 141.7703],
        [ 23.9307,  41.2577],
        [ 98.6786, 110.3586]], grad_fn=<AddmmBackward0>)

In [81]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [82]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.9159, 67.2859]], grad_fn=<AddmmBackward0>)

In [2]:
import matplotlib.pyplot as plt